- 기본정보 입력하기

In [ ]:
my_id = "user_id@poscohrd.com" # LP 아이디
start_day = "2022.09.01" # 조회할 교육시작 날짜
end_day = "2022.09.30" # 조회할 교육종료 날짜
company = "포스코" # 또는 "그룹사"
center_name = "C211005_21년 RPA 기초과정" # 교육과정 활동센터
category = "직무/혁신" # 직무분류 선택 
# (category 예시) 리더십/조직관리, 경영이념, 기술교육, 사외위탁, 상생, 안전교육, 위탁, 해외법인
employee_type = "일반" # 직원 유형 선택
# (employee_type 예시)국내직원, 그룹신입, 글로벌스탭 국내초청, 기술교육, 안전교육, 연봉제 일반직원, 연봉제 직책자, 연봉제 직책자 양성, 월급제, 위탁, 일반, 일반직원
# (employee_type 예시추가)임원, 직무일반(코칭), 직책자, 퇴직직원, 포스코 신입사원, 혁신, 혁신(QSS)
compulsory = "선택" # 선발, 필수, 희망, 연봉제 직책자 양성
course_name = "RPA 기초" # 조회할 교육과정명(※주의. 차수명아님)
edu_time = 16 # 교육과정 운영 시간
cost = 6500 # 교육과정 단가
finish = "수료" # 수료여부 선택 : 미수료, 미처리, 수료예상, 처리중
refund = "X" # 고용보험 환급여부 선택 : O
strange_point = "없음" # 교육생 특이사항 작성
department = "데이터혁신교육그룹" # 교육운영부서
charge = "유구봉" # 교육과정 담당자
edu_month = end_day.split(".")[1] 
edu_month = int(edu_month) # 데이터프레임을 엑셀파일로 저장시 파일이름으로 활용

- 패키지 불러오기

In [ ]:
from selenium import webdriver
import pyautogui as pg
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select

- LP 접속하기

In [ ]:
my_pw = pg.password(title= "LP_PW 입력", text = "PW를 입력하세요")

# 러닝플랫폼 접속하기
driver = webdriver.Chrome("../chromedriver") # 다른 폴더에 있을 경우 절대경로를 입력, 절대경로 = "저장폴더/chromedriver.exe"
driver.get("https://lp.posco.co.kr/S22/S22A10/member/login/loginForm.do")


# ID와 PW 입력하기
driver.find_element(By.ID, "email").send_keys(my_id)
driver.find_element(By.ID, "password").send_keys(my_pw)

# 로그인 클릭하기
driver.find_element(By.XPATH,'//*[@id="loginForm"]/div[2]/button').click()

- 팝업 닫기

In [ ]:
pg.sleep(3)
try:
      driver.find_elements(By.CLASS_NAME,'layer-btn-typeA')[1].click()
except:
      print('팝업이 없습니다. 계속 진행합니다')

- 관리자모드로 접속하기

In [ ]:
driver.find_element(By.XPATH, '//*[@id="btnMy"]').click()


# (방법1) "인재창조원 집합교육운영자" 선택하기
select = Select(driver.find_element(By.XPATH,'/html/body/header/div[2]/div[4]/select'))
select.select_by_value("C_00000001")

- "집합 차수관리" 클릭

In [ ]:
#클릭으로 안되서 url 직접 접속
url = 'https://lp.posco.co.kr/S22/S22A10/admin/course/courseCs.do'
driver.get(url)

- 조건 검색

In [ ]:
# 진행상태 "교육종료" 선택
select = Select(driver.find_element(By.ID, "eduStatusCode"))
select.select_by_value("F") # R = 교육준비중, O = 교육진행중, F = 교육종료

# 교육일 "직접입력" 선택
driver.find_element(By.XPATH, '//*[@id="searchForm"]/div/ul/li/input[4]').click()

pg.sleep(2)

# (운영과정에 맞춰 수정 필요)교육시작일 입력
driver.find_element(By.ID, "searchStartDt").send_keys(start_day)
pg.sleep(1)

# (운영과정에 맞춰 수정 필요)교육종료일 입력
driver.find_element(By.ID, "searchEndDt").send_keys(end_day)

# 검색할 "과정명" 선택
# select = Select(driver.find_element(By.ID, "keywordType"))
# select.select_by_value("N") # C = 차수명

# # (운영과정에 맞춰 수정 필요) "과정명" 입력
# driver.find_element(By.ID, "keyword").send_keys(course_name)

# 담당자 "전체" 선택
# driver.find_element_by_xpath('//*[@id="MyManager"]/option[contains(text(),"전체")]').click()
select = Select(driver.find_element(By.ID, "MyManager"))
select.select_by_value("")

# 검색 클릭
driver.find_element(By.ID, "btnView1").click()
pg.sleep(3)


- 데이터 개수 "500" 선택

In [ ]:
driver.find_element(By.XPATH, '//*[@id="grid_length"]/label/select/option[contains(text(),"500")]').click()
pg.sleep(3)

# 검색된 과정 개수 확인하기
row = driver.find_element(By.CSS_SELECTOR, "#grid > tbody > tr:nth-child(1) > td:nth-child(2)").text
row = int(row)
print("검색된 과정 개수: ", row)

- 차수별 설문 데이터 다운

In [ ]:
import pandas as pd

row = 3 #코드 완성후 삭제

# i = 10
# num = 0
# result_df = pd.DataFrame()

# for c in range(row):
for c in range(16,19):

# while num < row:
    # i = i + 1
    # print(i)
    pg.sleep(3)
    driver.find_element(By.ID, "btnSearch1{}".format(c+1)).click()# 해당차수 "운영관리" 선택
    pg.sleep(3)
 
    driver.find_element(By.XPATH, '//*[@id="tabForm"]/div/ul/li[11]/a').click()# 설문 선택

    try:
        driver.find_element(By.ID,'btnView2').click() #통계 클릭
        pg.sleep(1)
        driver.find_element(By.ID,'btnExlDwl').click() #엑셀 다운
        pg.sleep(5)
        driver.find_element(By.XPATH,'/html/body/div[6]/div[1]/button').click() #설문결과 닫기
    except:
        print('설문이 없습니다. 계속 진행합니다')

    pg.sleep(0.5)
    driver.back() #과정목록으로 돌아가기
    pg.sleep(0.5)
    driver.back() #과정목록으로 돌아가기
    pg.sleep(1)

    # num += 1

In [ ]:
import os
files_Path = "D:/Downloads/" # 파일들이 들어있는 폴더
file_name_and_time_lst = []
# 해당 경로에 있는 파일들의 생성시간을 함께 리스트로 넣어줌. 
for f_name in os.listdir(f"{files_Path}"):
      written_time = os.path.getctime(f"{files_Path}{f_name}")
      file_name_and_time_lst.append((f_name, written_time))
# 생성시간 역순으로 정렬하고, 
sorted_file_lst = sorted(file_name_and_time_lst, key=lambda x: x[1], reverse=True)
# 가장 앞에 이는 놈을 넣어준다.
recent_file = sorted_file_lst[0]
recent_file_name = recent_file[0]
print("최근 다운로드한 파일명: ", recent_file_name)

# 최근 다운로드 한 파일 판다스로 불러오기
df = pd.read_excel(r'D:\Downloads\{}'.format(recent_file_name))

In [ ]:
df2 = df.drop(index=1, axis=0) #1번 행 삭제
df2

In [ ]:
df2.at[0,'3분기'] = '555'  # 인덱스 4행, '3분기' 컬럼 값을 '555'로 변경

In [ ]:
df2.loc[(df['0'].str.contains("교육내용")), '0'] = "유익성"

In [ ]:
df2.rename(columns=df2.iloc[0], inplace=True) #0번행을 컬럼명으로 변경
df2.drop(index=0, axis=0, inplace=True) #0번 행 삭제

In [ ]:
driver.find_element(By.ID,'btnExlDwl').click()

In [ ]:
import pandas as pd


i = 10
num = 0
result_df = pd.DataFrame()
while num < row:
    i = i + 1
    print(i)
    pg.sleep(3)
    driver.find_element(By.ID, "btnSearch{}".format(i)).click()# 해당차수 "운영관리" 선택
    pg.sleep(3)
 
    driver.find_element(By.ID, '//*[@id="tabForm"]/div/ul/li[11]/a').click()# 설문 선택
    

- 여기서부터 수정


    driver.find_element(By.ID, "btnExlDwl1").click() # 명단 "엑셀 다운로드" 선택
    edu_name = driver.find_element(By.CSS_SELECTOR, "#courseCsName").text
    start_day = driver.find_element(By.CSS_SELECTOR, "#eduStartDt").text
    end_day = driver.find_element(By.CSS_SELECTOR, "#eduEndDt").text

    pyautogui.sleep(3)
    # 최근 다운로드한 파일명 확인하기
    import os
    files_Path = "D:/Downloads/" # 파일들이 들어있는 폴더
    file_name_and_time_lst = []
    # 해당 경로에 있는 파일들의 생성시간을 함께 리스트로 넣어줌. 
    for f_name in os.listdir(f"{files_Path}"):
        written_time = os.path.getctime(f"{files_Path}{f_name}")
        file_name_and_time_lst.append((f_name, written_time))
    # 생성시간 역순으로 정렬하고, 
    sorted_file_lst = sorted(file_name_and_time_lst, key=lambda x: x[1], reverse=True)
    # 가장 앞에 이는 놈을 넣어준다.
    recent_file = sorted_file_lst[0]
    recent_file_name = recent_file[0]
    print("최근 다운로드한 파일명: ", recent_file_name)

    # 최근 다운로드 한 파일 판다스로 불러오기
    df = pd.read_excel(r'D:\Downloads\{}'.format(recent_file_name))

    # 1행, 2행 삭제
    df2 = df.drop([0,1])
    
    # 차수명, 교육시작일, 교육종료일 입력하기
    df2["입과자 리스트"] = edu_name
    df2["Unnamed: 4"] = start_day
    df2["Unnamed: 5"] = end_day

    # 비어있는 데이터프레임에 데이터 추가하기
    result_df = pd.concat([result_df, df2])
   
    # 최근 다운로드 한 파일 삭제
    os.remove(files_Path+recent_file_name)

    # 웹페이지 뒤로가기
    pyautogui.hotkey("Alt", "left")

    # 페이지 새로고침
    pyautogui.press("F5")
    pyautogui.sleep(5)
    
    # 진행상태 "교육종료" 선택
    select = Select(driver.find_element(By.ID, "eduStatusCode"))
    select.select_by_value('F') # R = 교육준비중, O = 교육진행중, F = 교육종료

    # 교육일 "직접입력" 선택
    driver.find_element(By.XPATH, '//*[@id="searchForm"]/div/ul/li/input[4]').click()
    pyautogui.sleep(2)

    # 검색할 "과정명" 선택
    select = Select(driver.find_element(By.ID, "keywordType"))
    select.select_by_value('N') # C = 차수명

    # (운영과정에 맞춰 수정 필요) "과정명" 입력
    driver.find_element(By.ID, "keyword").send_keys(ci.course_name)
    pyautogui.sleep(1)

    # 담당자 "전체" 선택
    # driver.find_element_by_xpath('//*[@id="MyManager"]/option[contains(text(),"전체")]').click()
    select = Select(driver.find_element(By.ID, "MyManager"))
    select.select_by_value('')
    pyautogui.sleep(1)

    # 검색 클릭
    driver.find_element(By.ID, "btnView1").click()
    pyautogui.sleep(3)

    # 데이터 개수 "500" 선택
    driver.find_element(By.XPATH, '//*[@id="grid_length"]/label/select/option[contains(text(),"500")]').click()
    pyautogui.sleep(1)

    num += 1


In [ ]:
# 필요모듈 불러오기
import selenium
import time
from selenium import webdriver
import pyautogui
from selenium.webdriver.common.by import By
import course_info as ci

# ------------ cousre_info_data_start ---------------------------- #
# # 기본정보 입력하기
# my_id = "mk@poscohrd.com" # LP 아이디
# start_day = "2021.09.21" # 조회할 교육시작 날짜
# end_day = "2021.10.20" # 조회할 교육종료 날짜
# company = "포스코" # 그룹사
# center_name = "C211005_21년 RPA 기초과정" # 교육과정 활동센터
# category = "직무/혁신" # 리더십/조직관리, 경영이념, 기술교육, 사외위탁, 상생, 안전교육, 위탁, 해외법인
# employee_type = "일반" 
# # 국내직원, 그룹신입, 글로벌스탭 국내초청, 기술교육, 안전교육, 연봉제 일반직원, 연봉제 직책자, 연봉제 직책자 양성, 월급제, 위탁, 일반, 일반직원
# # 임원, 직무일반(코칭), 직책자, 퇴직직원, 포스코 신입사원, 혁신, 혁신(QSS)
# compulsory = "선택" # 선발, 필수, 희망, 연봉제 직책자 양성
# course_name = "RPA 기초" # 조회할 교육과정명(※주의. 차수명아님)
# edu_time = 16 # 교육과정 운영 시간
# cost = 6500 # 교육과정 단가
# finish = "수료" # 미수료, 미처리, 수료예상, 처리중
# refund = "X" # O
# strange_point = "없음"
# department = "데이터혁신교육그룹"
# charge = "박민경"
# edu_month = end_day.split(".")[1]
# edu_month = int(edu_month) # 최종 데이터프레임을 엑셀파일로 저장시 파일이름으로 활용
# ------------ cousre_info_data_end ---------------------------- #

# 러닝플랫폼 접속하기
driver = webdriver.Chrome("chromedriver") # 다른 폴더에 있을 경우 절대경로를 입력, 절대경로 = "저장폴더/chromedriver.exe"
driver.get("https://lp.posco.co.kr/S22/S22A10/member/login/loginForm.do")

# 브라우저 최대화
driver.minimize_window() # LP 창 최소화
windows = driver.window_handles # 최소화된 LP 창 windows 변수에 저장
driver.switch_to.window(windows[0]) # 최소화된 LP 창 windows 선택
driver.maximize_window() # 선택된 최소화된 LP 창 최대화

# LP_PW 설정하기
my_pw = pyautogui.password(title= "LP_PW 입력", text = "PW를 입력하세요")

# ID와 PW 입력하기
driver.find_element(By.ID, "email").send_keys(ci.my_id)
driver.find_element(By.ID, "password").send_keys(my_pw)

# 로그인 클릭하기
driver.find_element(By.XPATH,'//*[@id="loginForm"]/div[2]/button').click()

# 로그인 후 팝업창 뜨는 경우 닫기
btn_close = pyautogui.locateCenterOnScreen("close.PNG")
while btn_close != None:
    pyautogui.click(btn_close)
    btn_close = pyautogui.locateCenterOnScreen("close.PNG")

# 검색에 사용되는 클래스 불러오기
from selenium.webdriver.support.ui import Select
time.sleep(3)

# 관리자모드로 접속하기 전 프로필 선택하기
driver.find_element(By.XPATH, '/html/body/div[2]/header/div[2]/button[3]').click()

# (방법1) "인재창조원 집합교육운영자" 선택하기
select = Select(driver.find_element(By.XPATH,"/html/body/div[2]/header/div[2]/div[4]/select"))
select.select_by_value("C_00000001")

#(방법1) 과정관리-집합 차수관리 선택하기(마우스 좌표로 접속하기)
pyautogui.sleep(3)
pyautogui.position() # "과정관리" 버튼 위치(좌표) 확인하기
pyautogui.moveTo(65, 247) # "과정관리" 버튼에 마우스 가져다 놓기

pyautogui.sleep(3)
pyautogui.move(0, 40) # "과정관리" 하위 메뉴에서 "집합 차수관리" 버튼에 마우스 가져다 놓기

pyautogui.sleep(1)
pyautogui.click() # "집합 차수관리" 클릭
pyautogui.sleep(3)

# 검색조건 설정
# 진행상태 "교육종료" 선택
select = Select(driver.find_element(By.ID, "eduStatusCode"))
select.select_by_value("F") # R = 교육준비중, O = 교육진행중, F = 교육종료

# 교육일 "직접입력" 선택
driver.find_element(By.XPATH, '//*[@id="searchForm"]/div/ul/li/input[4]').click()

pyautogui.sleep(2)

# (운영과정에 맞춰 수정 필요)교육시작일 입력
driver.find_element(By.ID, "searchStartDt").send_keys(ci.start_day)
pyautogui.sleep(1)

# (운영과정에 맞춰 수정 필요)교육종료일 입력
driver.find_element(By.ID, "searchEndDt").send_keys(ci.end_day)

# 검색할 "과정명" 선택
select = Select(driver.find_element(By.ID, "keywordType"))
select.select_by_value("N") # C = 차수명

# (운영과정에 맞춰 수정 필요) "과정명" 입력
driver.find_element(By.ID, "keyword").send_keys(ci.course_name)

# 담당자 "전체" 선택
# driver.find_element_by_xpath('//*[@id="MyManager"]/option[contains(text(),"전체")]').click()
select = Select(driver.find_element(By.ID, "MyManager"))
select.select_by_value("")

# 검색 클릭
driver.find_element(By.ID, "btnView1").click()
pyautogui.sleep(3)

# 데이터 개수 "500" 선택
driver.find_element(By.XPATH, '//*[@id="grid_length"]/label/select/option[contains(text(),"500")]').click()
pyautogui.sleep(3)

# 검색된 과정 개수 확인하기
row = driver.find_element(By.CSS_SELECTOR, "#grid > tbody > tr:nth-child(1) > td:nth-child(2)").text
row = int(row)
print("검색된 과정 개수: ", row)

import pandas as pd

# 차수별 명단 선택
i = 10
num = 0
result_df = pd.DataFrame()
while num < row:
    i = i + 1
    print(i)
    pyautogui.sleep(3)
    driver.find_element(By.ID, "btnSearch{}".format(i)).click()# 해당차수 "운영관리" 선택
    pyautogui.sleep(3)
    driver.find_element(By.XPATH, '//*[@id="grid_length"]/label/select/option[contains(text(),"2000")]').click() #데이터 개수 "2000" 선택
    pyautogui.sleep(3)
    driver.find_element(By.ID, "btnExlDwl1").click() # 명단 "엑셀 다운로드" 선택
    edu_name = driver.find_element(By.CSS_SELECTOR, "#courseCsName").text
    start_day = driver.find_element(By.CSS_SELECTOR, "#eduStartDt").text
    end_day = driver.find_element(By.CSS_SELECTOR, "#eduEndDt").text

    pyautogui.sleep(3)
    # 최근 다운로드한 파일명 확인하기
    import os
    files_Path = "D:/Downloads/" # 파일들이 들어있는 폴더
    file_name_and_time_lst = []
    # 해당 경로에 있는 파일들의 생성시간을 함께 리스트로 넣어줌. 
    for f_name in os.listdir(f"{files_Path}"):
        written_time = os.path.getctime(f"{files_Path}{f_name}")
        file_name_and_time_lst.append((f_name, written_time))
    # 생성시간 역순으로 정렬하고, 
    sorted_file_lst = sorted(file_name_and_time_lst, key=lambda x: x[1], reverse=True)
    # 가장 앞에 이는 놈을 넣어준다.
    recent_file = sorted_file_lst[0]
    recent_file_name = recent_file[0]
    print("최근 다운로드한 파일명: ", recent_file_name)

    # 최근 다운로드 한 파일 판다스로 불러오기
    df = pd.read_excel(r'D:\Downloads\{}'.format(recent_file_name))

    # 1행, 2행 삭제
    df2 = df.drop([0,1])
    
    # 차수명, 교육시작일, 교육종료일 입력하기
    df2["입과자 리스트"] = edu_name
    df2["Unnamed: 4"] = start_day
    df2["Unnamed: 5"] = end_day

    # 비어있는 데이터프레임에 데이터 추가하기
    result_df = pd.concat([result_df, df2])
   
    # 최근 다운로드 한 파일 삭제
    os.remove(files_Path+recent_file_name)

    # 웹페이지 뒤로가기
    pyautogui.hotkey("Alt", "left")

    # 페이지 새로고침
    pyautogui.press("F5")
    pyautogui.sleep(5)
    
    # 진행상태 "교육종료" 선택
    select = Select(driver.find_element(By.ID, "eduStatusCode"))
    select.select_by_value('F') # R = 교육준비중, O = 교육진행중, F = 교육종료

    # 교육일 "직접입력" 선택
    driver.find_element(By.XPATH, '//*[@id="searchForm"]/div/ul/li/input[4]').click()
    pyautogui.sleep(2)

    # 검색할 "과정명" 선택
    select = Select(driver.find_element(By.ID, "keywordType"))
    select.select_by_value('N') # C = 차수명

    # (운영과정에 맞춰 수정 필요) "과정명" 입력
    driver.find_element(By.ID, "keyword").send_keys(ci.course_name)
    pyautogui.sleep(1)

    # 담당자 "전체" 선택
    # driver.find_element_by_xpath('//*[@id="MyManager"]/option[contains(text(),"전체")]').click()
    select = Select(driver.find_element(By.ID, "MyManager"))
    select.select_by_value('')
    pyautogui.sleep(1)

    # 검색 클릭
    driver.find_element(By.ID, "btnView1").click()
    pyautogui.sleep(3)

    # 데이터 개수 "500" 선택
    driver.find_element(By.XPATH, '//*[@id="grid_length"]/label/select/option[contains(text(),"500")]').click()
    pyautogui.sleep(1)

    num += 1

# ---------------------------- 데이터 전처리 과정 ---------------------------- #
# 필요한 열(21개)만 남기고, 나머지 열 삭제
i = 21
col_list = []
while i <= 25: 
    col_list.append("Unnamed: {}".format(i)) # 나머지 열 list로 만들기
    i += 1
df = result_df.drop(columns=col_list) # 나머지 열 삭제

# columns 명칭을 변경하기(교육수료현황에 있는 행값ㅇ 입력)
result_dict = {}
new_col = ["교육과정명", "성명", "삭제열", "직번", "교육기간From", "교육기간To", "포스코/그룹사", "활동센터", "분류", "유형", "필수구분", "교육과정명_통합", "총 교육시간", "시간당표준단가", "회사구분코드", "학습자 소속 회사명", "수료여부", "고용보험 환급여부", "교육과정 중 특이사항", "운영부서", "운영담당자"]
for idx, col in enumerate(df.columns):
    result_dict[col] = new_col[idx]

# columns 명칭 변경
df2 = df.rename(columns=result_dict)

# 삭제열 삭제
df3 = df2.drop("삭제열", axis=1)

# columns 위치 변경
df4 = df3[["포스코/그룹사", "활동센터", "분류", "유형", "필수구분", "교육과정명_통합", "교육과정명", "교육기간From", "교육기간To", "총 교육시간", "시간당표준단가","회사구분코드", "학습자 소속 회사명", "성명", "직번", "수료여부", "고용보험 환급여부", "교육과정 중 특이사항", "운영부서", "운영담당자"]]

# coloumns의 데이터 변경하기
cv = ["포스코/그룹사", "활동센터", "분류", "유형", "필수구분", "교육과정명_통합", "총 교육시간", "시간당표준단가","회사구분코드", "학습자 소속 회사명", "수료여부", "고용보험 환급여부", "교육과정 중 특이사항", "운영부서", "운영담당자"]
val = [ci.company, ci.center_name, ci.category, ci.employee_type, ci.compulsory, ci.course_name, ci.edu_time, ci.cost, 1, "(주)포스코", ci.finish, ci.refund, ci.strange_point, ci.department, ci.charge]

df4[cv] = val

# 데이터프레임 엑셀 저장하기
df4.to_excel("{}월 {} 교육수료현황.xlsx".format(ci.edu_month, ci.course_name))
print("엑셀파일 저장 완료했습니다.")